In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time, os, requests
from PIL import Image
from io import BytesIO

def google_image_crawl(query, max_images=20, save_dir='downloads'):
    # 옵션 설정
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--no-sandbox')

    # 드라이버 경로 설정 (필요시 절대경로로 수정)
    driver = webdriver.Chrome(options=chrome_options)

    # 쿼리 인코딩
    search_url = f"https://www.google.com/search?tbm=isch&q={query}"
    driver.get(search_url)

    # 이미지 클릭 없이 스크롤하면서 URL 수집
    image_urls = set()
    last_height = driver.execute_script("return document.body.scrollHeight")

    while len(image_urls) < max_images:
        thumbnails = driver.find_elements(By.CSS_SELECTOR, "img.rg_i")

        for img in thumbnails[len(image_urls):]:
            try:
                img.click()
                time.sleep(1)
                images = driver.find_elements(By.CSS_SELECTOR, "img.n3VNCb")
                for image in images:
                    src = image.get_attribute("src")
                    if src and "http" in src:
                        image_urls.add(src)
                        if len(image_urls) >= max_images:
                            break
            except Exception:
                print("Error clicking image, skipping...")
                continue

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    driver.quit()

    # 이미지 저장
    os.makedirs(save_dir, exist_ok=True)
    for i, url in enumerate(image_urls):
        try:
            response = requests.get(url, timeout=10)
            image = Image.open(BytesIO(response.content))
            ext = image.format.lower()
            file_path = os.path.join(save_dir, f"{query}_{i}.{ext}")
            image.save(file_path)
            print(f"Saved {file_path}")
        except Exception as e:
            print(f"Failed to save image {i}: {e}")

# 사용 예시
google_image_crawl("cat", max_images=10)
